<a href="https://colab.research.google.com/github/trewto/Yolo-Practices/blob/main/yolo_paper_updated_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytube opencv-python


In [4]:
!pip install pytybe

ERROR: Could not find a version that satisfies the requirement pytybe (from versions: none)
ERROR: No matching distribution found for pytybe


In [5]:
!pip install --upgrade pytube

In [10]:
!pip install yt-dlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.1/170.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 78.3 MB/s eta 0:00:00


In [11]:
import yt_dlp

# YouTube video URL
youtube_url = 'https://www.youtube.com/watch?v=n6ShWPom7ys'

# Define options for yt-dlp
ydl_opts = {
    'format': 'bestvideo+bestaudio/best',
    'outtmpl': '/content/video.mp4',  # Save video as 'video.mp4' in the Colab environment
    'noplaylist': True,  # Download only the single video, not the whole playlist
}

# Download the video
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([youtube_url])

print("Download completed.")


[youtube] Extracting URL: https://www.youtube.com/watch?v=n6ShWPom7ys
[youtube] n6ShWPom7ys: Downloading webpage
[youtube] n6ShWPom7ys: Downloading ios player API JSON
[youtube] n6ShWPom7ys: Downloading web creator player API JSON
[youtube] n6ShWPom7ys: Downloading player 5f8f5b0f
[youtube] n6ShWPom7ys: Downloading m3u8 information
[youtube] n6ShWPom7ys: Downloading MPD manifest
[info] n6ShWPom7ys: Downloading 1 format(s): 137+251
[download] Destination: /content/video.mp4.f137.mp4
[download] 100% of  932.13MiB in 00:05:25 at 2.87MiB/s   
[download] Destination: /content/video.mp4.f251.webm
[download] 100% of   26.34MiB in 00:00:07 at 3.31MiB/s   
[Merger] Merging formats into "/content/video.mp4.mkv"
Deleting original file /content/video.mp4.f137.mp4 (pass -k to keep)
Deleting original file /content/video.mp4.f251.webm (pass -k to keep)
Download completed.


In [13]:
import os

# Check if the file exists
file_path = '/content/video.mp4.mkv'
if os.path.exists(file_path):
    print(f"Video downloaded successfully: {file_path}")
else:
    print("Video download failed.")


Video downloaded successfully: /content/video.mp4.mkv


In [14]:
!pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.8/871.8 kB 22.5 MB/s eta 0:00:00


In [15]:
!pip install supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 7.6 MB/s eta 0:00:00


In [23]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO
import supervision as sv

# Define the line coordinates
LINE_START = (500, 2)
LINE_END = (500, 718)

# Load the YOLO model
model = YOLO("best.pt")

# Path to the downloaded video
SOURCE_VIDEO_PATH = '/content/video.mp4.mkv'

# Initialize video capture
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
track_history = defaultdict(lambda: [])

line_crossing_counts = defaultdict(int)

def two_line_intersect(x1, y1, x2, y2, x3, y3, x4, y4):
    if x1 == x2 and x3 == x4:
        return False
    elif x1 == x2:
        x_intersect = x1
        m2 = (y4 - y3) / (x4 - x3)
        c2 = y3 - m2 * x3
        y_intersect = m2 * x_intersect + c2
    elif x3 == x4:
        x_intersect = x3
        m1 = (y2 - y1) / (x2 - x1)
        c1 = y1 - m1 * x1
        y_intersect = m1 * x_intersect + c1
    else:
        m1 = (y2 - y1) / (x2 - x1)
        m2 = (y4 - y3) / (x4 - x3)
        if m1 == m2:
            return False
        x_intersect = (c2 - c1) / (m1 - m2)
        y_intersect = m1 * x_intersect + c1

    if (min(x1, x2) <= x_intersect <= max(x1, x2)) and (min(x3, x4) <= x_intersect <= max(x3, x4)) and \
       (min(y1, y2) <= y_intersect <= max(y1, y2)) and (min(y3, y4) <= y_intersect <= max(y3, y4)):
        return True
    return False

frame_count = 1
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

with sv.VideoSink("output_single_line.mp4", video_info) as sink:
    while cap.isOpened():
        success, frame = cap.read()
        frame_count += 2
        if frame_count % 500 == 0 or frame_count % 501 == 0:
            print(frame_count)

        if frame_count % 5000 == 0 or frame_count % 5001 == 0:
            break

        if not success:
            print("Error: Could not read frame.")
            break

        if success:
            results = model.track(frame, persist=True, verbose=False)
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id
            if track_ids is not None:
                track_ids = track_ids.int().cpu().tolist()
                annotated_frame = results[0].plot()

                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    track = track_history[track_id]
                    track.append((int(x), int(y)))

                    if len(track) > 30:
                        track.pop(0)

                    points = np.array(track).astype(np.int32).reshape((-1, 1, 2))
                    cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=2)

                    if len(track) > 1:
                        prev_x = track[-2][0]
                        prev_y = track[-2][1]

                        if two_line_intersect(x, y, prev_x, prev_y, LINE_START[0], LINE_START[1], LINE_END[0], LINE_END[1]):
                            line_crossing_counts[track_id] = 1
                            #print(f"Object {track_id} crossed the line. Count: {line_crossing_counts[track_id]}")

                    cv2.putText(annotated_frame, f"Total: {sum(line_crossing_counts.values())} , Frame = {frame_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    cv2.line(annotated_frame, LINE_START, LINE_END, (0, 0, 255), 2)

                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
            else:
                x = 1
                #print("Error x= 1")

        else:
            break

        sink.write_frame(annotated_frame)

cap.release()
cv2.destroyAllWindows()
print(f"Total objects crossed the line: {sum(line_crossing_counts.values())}")
print("Processing complete.")


501
1503
2505
3507
4509
Total objects crossed the line: 16
Processing complete.
